# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rorvik,64.8621,11.2373,3.99,100,20,1.54,NO,1680920132
1,1,katsuura,35.1333,140.3000,18.85,50,5,11.25,JP,1680920309
2,2,rikitea,-23.1203,-134.9692,25.00,68,49,6.17,PF,1680920293
3,3,carnarvon,-24.8667,113.6333,22.80,55,8,5.08,AU,1680920310
4,4,hami,42.8000,93.4500,11.87,12,6,2.40,CN,1680920310


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 700,
    frame_height = 500
)

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [79]:
# Narrow down cities that fit criteria and drop any results with null values
desired_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & 
                                   (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] <= 10)]

# Drop any rows with null values
desired_city_df = desired_city_df.dropna(how='any')

# Display sample data
desired_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,dum duma,27.5667,95.5667,24.18,49,5,1.38,IN,1680920313
22,22,kankon,15.0167,74.0167,26.57,71,7,1.96,IN,1680920314
34,34,coahuayana,18.7333,-103.6833,22.56,77,0,0.76,MX,1680920316
82,82,cabo san lucas,22.8909,-109.9124,24.78,47,0,1.54,MX,1680920325
115,115,cozumel,20.5083,-86.9458,26.64,80,0,3.13,MX,1680920331
144,144,vao,-22.6667,167.4833,24.32,75,2,3.55,NC,1680920336
157,157,beira,-19.8436,34.8389,22.05,100,0,2.06,MZ,1680920339
201,201,inhambane,-23.8650,35.3833,21.11,88,0,4.10,MZ,1680920346
230,230,londa,15.4691,74.5191,22.63,60,10,1.79,IN,1680920352
262,262,tomatlan,19.9333,-105.2500,21.44,68,0,1.05,MX,1680920359


### Step 3: Create a new DataFrame called `hotel_df`.

In [80]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = desired_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", "")

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,dum duma,IN,27.5667,95.5667,49,
22,kankon,IN,15.0167,74.0167,71,
34,coahuayana,MX,18.7333,-103.6833,77,
82,cabo san lucas,MX,22.8909,-109.9124,47,
115,cozumel,MX,20.5083,-86.9458,80,
144,vao,NC,-22.6667,167.4833,75,
157,beira,MZ,-19.8436,34.8389,100,
201,inhambane,MZ,-23.8650,35.3833,88,
230,londa,IN,15.4691,74.5191,60,
262,tomatlan,MX,19.9333,-105.2500,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [81]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "limit":limit,
    "type":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df["Lng"]
    latitude = hotel_df["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = "circle:" + str(longitude[index]) + "," + str(latitude[index]) + "," + str(radius)
    params["bias"] = "proximity:" + str(longitude[index]) + "," + str(latitude[index])
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dum duma - nearest hotel: No hotel found
kankon - nearest hotel: Dreamcatcher
coahuayana - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
cozumel - nearest hotel: Hotel Isla Esmeralda
vao - nearest hotel: Nataiwatch
beira - nearest hotel: Hotel Savoy
inhambane - nearest hotel: Hotel de Inhambane
londa - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
caravelas - nearest hotel: Pousada dos Navegantes
buin - nearest hotel: Hotel Casa Real
camargo - nearest hotel: Hotel Santa Fe
inverell - nearest hotel: Oxford Hotel
kotri - nearest hotel: Shah Ji Hotel
matamoros - nearest hotel: No hotel found
poum - nearest hotel: No hotel found
tombouctou - nearest hotel: No hotel found
safaga - nearest hotel: Dolphin Hotel
araouane - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
20,dum duma,IN,27.5667,95.5667,49,No hotel found
22,kankon,IN,15.0167,74.0167,71,Dreamcatcher
34,coahuayana,MX,18.7333,-103.6833,77,No hotel found
82,cabo san lucas,MX,22.8909,-109.9124,47,Comfort Rooms
115,cozumel,MX,20.5083,-86.9458,80,Hotel Isla Esmeralda
144,vao,NC,-22.6667,167.4833,75,Nataiwatch
157,beira,MZ,-19.8436,34.8389,100,Hotel Savoy
201,inhambane,MZ,-23.8650,35.3833,88,Hotel de Inhambane
230,londa,IN,15.4691,74.5191,60,No hotel found
262,tomatlan,MX,19.9333,-105.2500,68,Hotel Hacienda Vieja Tomatlan


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [82]:
%%capture --no-display

# Configure the map plot
desired_city_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
    frame_width = 700,
    frame_height = 500
)

# Display the map
desired_city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)